In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from ydata_profiling import ProfileReport
import numpy as np
import pandas as pd
import dic_moledo
from pyspark.sql.functions import col, create_map, lit
from itertools import chain
import importlib

importlib.reload(dic_moledo)

<module 'dic' from 'c:\\Users\\mathe\\OneDrive\\Documentos\\Códigos\\RECONHECIMENTO-DE-PADROES-COM-INTELIGENCIA-ARTIFICIAL-MICRODADOS-ENEM\\dic.py'>

In [50]:
# Configura a SparkSession
spark = SparkSession.builder \
    .appName("Minha Aplicação") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.executor.memoryOverhead", "1g") \
    .getOrCreate()

# Agora você pode verificar todas as configurações atuais
conf = spark.sparkContext.getConf()
print({item[0]: item[1] for item in conf.getAll()})



{'spark.driver.extraJavaOptions': '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false', 'spark.driver.memory': '4g', 'spark.executor.memory': '4g', 'spark.driver.host': 'DESKTOP-D8Q42PQ', 'spark.

In [51]:
# Lendo um arquivo CSV
df = spark.read.csv("MICRODADOS_ENEM_2022.csv", header=True, sep=";")

In [52]:
colunas_descartadas = ['NU_INSCRICAO', 'NU_ANO', 'TP_ENSINO','CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 
                       'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT','TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 
                       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT', 'NU_NOTA_COMP1', 
                       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5','TP_SEXO','TP_ESTADO_CIVIL','Q011','Q005','Q007','Q008','Q009','Q010','Q012','Q013','Q015',
                       'Q016','Q017','Q018','Q019','Q020','Q021','Q023','Q014','Q022','Q025','TP_ST_CONCLUSAO','TP_FAIXA_ETARIA','IN_TREINEIRO']

# Descartando colunas
df = df.drop(*colunas_descartadas)

df.show()

+---------------+-----------+----------------+---------------+---------------+---------+------------+-----------+-----------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+-----------------+---------------+----+----+----+----+----+----+
|TP_FAIXA_ETARIA|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|IN_TREINEIRO|CO_UF_PROVA|SG_UF_PROVA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|TP_LINGUA|TP_STATUS_REDACAO|NU_NOTA_REDACAO|Q001|Q002|Q003|Q004|Q006|Q024|
+---------------+-----------+----------------+---------------+---------------+---------+------------+-----------+-----------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+-----------------+---------------+----+----+----+----+----+----+
|             14|          2|               1|              1|              2|        1|

In [74]:
# Descartando linhas que contem valores diferentes de 1 em TP_PRESENCA_CN
df = df.filter(df.TP_PRESENCA_CN == 1)
df = df.filter(df.TP_PRESENCA_CH == 1)
df = df.filter(df.TP_PRESENCA_LC == 1)
df = df.filter(df.TP_PRESENCA_MT == 1)

df.count()

# Descartando colunas que contem valores H na coluna Q001 e Q002
df = df.filter(df.Q001 != 'H')
df = df.filter(df.Q002 != 'H')

df.count()

AttributeError: 'DataFrame' object has no attribute 'TP_PRESENCA_CN'

In [54]:
# Criando a coluna media e descartando as colunas de notas
df = df.withColumn('NU_NOTA_MEDIA', (col('NU_NOTA_CN') + col('NU_NOTA_CH') + col('NU_NOTA_LC') + col('NU_NOTA_MT') + col('NU_NOTA_REDACAO'))/5)
df = df.drop('NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT')

df.show()

+---------------+-----------+----------------+---------------+---------------+---------+------------+-----------+-----------+--------------+--------------+--------------+--------------+---------+-----------------+---------------+----+----+----+----+----+----+------------------+
|TP_FAIXA_ETARIA|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|IN_TREINEIRO|CO_UF_PROVA|SG_UF_PROVA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|TP_LINGUA|TP_STATUS_REDACAO|NU_NOTA_REDACAO|Q001|Q002|Q003|Q004|Q006|Q024|     NU_NOTA_MEDIA|
+---------------+-----------+----------------+---------------+---------------+---------+------------+-----------+-----------+--------------+--------------+--------------+--------------+---------+-----------------+---------------+----+----+----+----+----+----+------------------+
|              5|          2|               1|              1|              2|        1|           0|         29|         BA|             1|             1|        

In [55]:
colunas_descartadas = ['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT','TP_STATUS_REDACAO','SG_UF_PROVA','NU_NOTA_REDACAO','CO_UF_PROVA','TP_ANO_CONCLUIU','TP_NACIONALIDADE']

# Descartando colunas
df = df.drop(*colunas_descartadas)


In [56]:
print(dic.q006_dict)

{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17}


In [57]:
# Colunas Q001 ate Q025
colunas = ['Q001','Q002','Q003','Q004','Q006','Q024']


for coluna in colunas:
    # Pegando os valores correspondentes a cada letra no dicionario dic
    nome_dict = coluna.lower() + '_dict'  # Constrói o nome do dicionário
    if hasattr(dic_moledo, nome_dict):  # Verifica se o dicionário existe no módulo dicionario
        mapeamento = getattr(dic_moledo, nome_dict)  # Obtém o dicionário
        mapeamento_expr = create_map([lit(x) for x in chain(*mapeamento.items())])
        df = df.withColumn(coluna, mapeamento_expr.getItem(col(coluna)))


In [58]:
df.show()

+---------------+-----------+---------------+---------+------------+---------+----+----+----+----+----+----+------------------+
|TP_FAIXA_ETARIA|TP_COR_RACA|TP_ST_CONCLUSAO|TP_ESCOLA|IN_TREINEIRO|TP_LINGUA|Q001|Q002|Q003|Q004|Q006|Q024|     NU_NOTA_MEDIA|
+---------------+-----------+---------------+---------+------------+---------+----+----+----+----+----+----+------------------+
|              5|          2|              1|        1|           0|        1|   5|   6|   1|   4|   2|   1|            558.24|
|              6|          3|              1|        1|           0|        1|   3|   1|   1|   2|   1|   2|            394.62|
|              6|          2|              1|        1|           0|        1|   5|   5|   2|   2|   2|   1|             414.1|
|              4|          3|              1|        1|           0|        1|   5|   5|   2|   2|   2|   1|             438.1|
|              2|          1|              2|        3|           0|        1|   5|   5|   2|   1|   2| 

In [59]:
# Alterando o tipo de dado de string para int 
df = df.withColumn("TP_FAIXA_ETARIA", df["TP_FAIXA_ETARIA"].cast("int"))
df = df.withColumn("TP_COR_RACA", df["TP_COR_RACA"].cast("int"))
df = df.withColumn("TP_ST_CONCLUSAO", df["TP_ST_CONCLUSAO"].cast("int"))
df = df.withColumn("TP_ESCOLA", df["TP_ESCOLA"].cast("int"))
df = df.withColumn("IN_TREINEIRO", df["IN_TREINEIRO"].cast("int"))
df = df.withColumn("TP_LINGUA", df["TP_LINGUA"].cast("int"))

In [61]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# Convert the Spark DataFrame back to a pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()
result_pdf.dtypes

C:\Users\mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\sql\pandas\conversion.py:111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 4.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


TP_FAIXA_ETARIA      int32
TP_COR_RACA          int32
TP_ST_CONCLUSAO      int32
TP_ESCOLA            int32
IN_TREINEIRO         int32
TP_LINGUA            int32
Q001                 int32
Q002                 int32
Q003                 int32
Q004                 int32
Q006                 int32
Q024                 int32
NU_NOTA_MEDIA      float64
dtype: object

In [62]:
result_pdf = pd.get_dummies(result_pdf)
result_pdf.head()

,TP_FAIXA_ETARIA,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,IN_TREINEIRO,TP_LINGUA,Q001,Q002,Q003,Q004,Q006,Q024,NU_NOTA_MEDIA
0,5,2,1,1,0,1,5,6,1,4,2,1,558.24
1,6,3,1,1,0,1,3,1,1,2,1,2,394.62
2,6,2,1,1,0,1,5,5,2,2,2,1,414.10
3,4,3,1,1,0,1,5,5,2,2,2,1,438.10
4,2,1,2,3,0,1,5,5,2,1,2,1,576.70


In [63]:
profile = ProfileReport(result_pdf, title="Pandas Profiling Report")
profile.to_file("output_q006_reload.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 111.01it/s]


In [66]:
df.show()


+---------------+-----------+---------------+---------+------------+---------+----+----+----+----+----+----+------------------+
|TP_FAIXA_ETARIA|TP_COR_RACA|TP_ST_CONCLUSAO|TP_ESCOLA|IN_TREINEIRO|TP_LINGUA|Q001|Q002|Q003|Q004|Q006|Q024|     NU_NOTA_MEDIA|
+---------------+-----------+---------------+---------+------------+---------+----+----+----+----+----+----+------------------+
|              5|          2|              1|        1|           0|        1|   5|   6|   1|   4|   2|   1|            558.24|
|              6|          3|              1|        1|           0|        1|   3|   1|   1|   2|   1|   2|            394.62|
|              6|          2|              1|        1|           0|        1|   5|   5|   2|   2|   2|   1|             414.1|
|              4|          3|              1|        1|           0|        1|   5|   5|   2|   2|   2|   1|             438.1|
|              2|          1|              2|        3|           0|        1|   5|   5|   2|   1|   2| 

In [69]:
# Contando a quantidade de linhas valores nulos

for coluna in df.columns:
    print(coluna, df.filter(col(coluna).isNull()).count())

TP_FAIXA_ETARIA 0
TP_COR_RACA 0
TP_ST_CONCLUSAO 0
TP_ESCOLA 0
IN_TREINEIRO 0
TP_LINGUA 0
Q001 0
Q002 0
Q003 0
Q004 0
Q006 0
Q024 0
NU_NOTA_MEDIA 0


In [73]:
df.createOrReplaceTempView("df")

df_consulta = spark.sql("SELECT Q001, COUNT(*) FROM df GROUP BY 1 ORDER BY 1")
df_consulta.show()

+----+--------+
|Q001|count(1)|
+----+--------+
|   1|   75867|
|   2|  350284|
|   3|  291859|
|   4|  259673|
|   5|  720913|
|   6|  250540|
|   7|  193050|
|   8|  202637|
+----+--------+



In [72]:
df.createOrReplaceTempView("df")

df_consulta = spark.sql("SELECT Q002, COUNT(*) FROM df GROUP BY 1 ORDER BY 1")
df_consulta.show()

+----+--------+
|Q002|count(1)|
+----+--------+
|   1|   46006|
|   2|  238683|
|   3|  232165|
|   4|  262334|
|   5|  851162|
|   6|  329384|
|   7|  322603|
|   8|   62486|
+----+--------+



In [65]:
df.createOrReplaceTempView("df")

# Executando uma query SQL
df_sul_nota = spark.sql("SELECT AVG(NU_NOTA_MEDIA) SUL FROM df WHERE CO_UF_PROVA in ('41', '42', '43')")
df_sudeste_nota = spark.sql("SELECT AVG(NU_NOTA_MEDIA) SUDESTE FROM df WHERE CO_UF_PROVA in ('31', '32', '33', '35')")
df_norte_nota = spark.sql("SELECT AVG(NU_NOTA_MEDIA) NORTE FROM df WHERE CO_UF_PROVA in ('11', '12', '13', '14', '15', '16', '17')")
df_nordeste_nota = spark.sql("SELECT AVG(NU_NOTA_MEDIA) NORDESTE FROM df WHERE CO_UF_PROVA in ('21', '22', '23', '24', '25', '26', '27')")
df_centro_oeste_nota = spark.sql("SELECT AVG(NU_NOTA_MEDIA) CENTRO_OESTE FROM df WHERE CO_UF_PROVA in ('50', '51', '52', '53')")

df_sul_nota.show()
df_sudeste_nota.show()
df_norte_nota.show()
df_nordeste_nota.show()






AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `CO_UF_PROVA` cannot be resolved. Did you mean one of the following? [`TP_COR_RACA`, `TP_ESCOLA`, `Q001`, `Q002`, `Q003`].; line 1 pos 44;
'Project ['AVG('NU_NOTA_MEDIA) AS SUL#3571]
+- 'Filter 'CO_UF_PROVA IN (41,42,43)
   +- SubqueryAlias df
      +- View (`df`, [TP_FAIXA_ETARIA#3359,TP_COR_RACA#3373,TP_ST_CONCLUSAO#3387,TP_ESCOLA#3401,IN_TREINEIRO#3415,TP_LINGUA#3429,Q001#3201,Q002#3215,Q003#3229,Q004#3243,Q006#3257,Q024#3271,NU_NOTA_MEDIA#3018])
         +- Project [TP_FAIXA_ETARIA#3359, TP_COR_RACA#3373, TP_ST_CONCLUSAO#3387, TP_ESCOLA#3401, IN_TREINEIRO#3415, cast(TP_LINGUA#2712 as int) AS TP_LINGUA#3429, Q001#3201, Q002#3215, Q003#3229, Q004#3243, Q006#3257, Q024#3271, NU_NOTA_MEDIA#3018]
            +- Project [TP_FAIXA_ETARIA#3359, TP_COR_RACA#3373, TP_ST_CONCLUSAO#3387, TP_ESCOLA#3401, cast(IN_TREINEIRO#2684 as int) AS IN_TREINEIRO#3415, TP_LINGUA#2712, Q001#3201, Q002#3215, Q003#3